# Binary Search and Information

A visualisation of how information measured in bits coresponds to asking binary (yes/no) questions.

In [ ]:
import numpy as np

A random variable (RV) $X$ is uniformly distributed over the integers ${\mathcal{X} = \{0, 1, \dots, M-1\}}$.
The RV $Y$ is obtained by transmitting $X$ over a channel that adds the independent RV $\Delta \sim \text{Unif}(\{0, 1, \dots, m-1\})$ to $X$ and takes the result modulo $M$, i.e., ${Y = (X + \Delta) \mod M}$.
It is given, that $M > m$.

In [ ]:
class RandomSource():
    def __init__(self, m: int, M: int):
        self.m = m
        self.M = M
        self.rng = np.random.default_rng()
        self.resample()

    def resample(self):
        self.x = self.rng.integers(0, self.M)
        self.delta = self.rng.integers(0, self.m)
        self.y = (self.x + self.delta) % self.M
    
    def observe_y(self) -> int:
        return self.y
    
    def is_x_geq(self, num: int) -> bool:
        """ Return true if x is greater than or equal to `num` """
        return self.x >= num
    
    def check_x_eq(self, num: int):
        """ Prints success message if `num` equals x """
        if num == self.x:
            print(f'Success you found x = {num}!')
        else:
            print(f'No x is not {num}.')

src = RandomSource(8, 128)

Binary search is an algorithm to find the entry equal to $x$ in a sorted array. In each iteration the middle entry of the array is examined. By finding wether it is $\geq x$, the search space can be restricted to the right or left half in the next iteration. For details see https://en.wikipedia.org/wiki/Binary_search

Here we use a binary search style technique when asking our source the binary (yes/no) question: Is $x \geq \texttt{num}$?
Counting the number of questions we must ask gives us a measure of how much information we must gain to know $x$.

In [ ]:
def binary_search(src: RandomSource, possible_x: np.ndarray) -> int:
    """ Returns the number of binary questions required to find x """
    num_possible = len(possible_x)
    possible_x = np.sort(possible_x)

    count_questions = 0
    while len(possible_x) > 1:
        middle_idx = len(possible_x) // 2
        num = possible_x[middle_idx]

        count_questions += 1
        if src.is_x_geq(num):
            possible_x = possible_x[middle_idx:]
        else:
            possible_x = possible_x[:middle_idx]

    assert len(possible_x) == 1 # otherwise while loop would not have stopped (ignore possibiltiy that `x` not in `possible_x`)

    print(f'Binary search in {num_possible} possible values:', end=' ')
    src.check_x_eq(possible_x[0])

    return count_questions

Define the probabilities $P(x)$, $P(y)$, $P(x|y)$, and $P(x,y)$ for later use.

In [ ]:
P_x = np.ones((src.M, 1)) / src.M                               # P(x) = 1/M  forall x
P_y = np.ones((1, src.M)) / src.M                               # P(y) = 1/M  forall y
P_x_given_y = np.zeros((src.M, src.M))
for y in range(src.M):
    delta = np.arange(src.m)
    possible_x = (y - delta) % src.m
    P_x_given_y[possible_x, y] = np.ones(src.m) / src.m         # P(x|y) = 1/m  forall x in X(y) = {(y - delta) % M: delta in {0,...,m-1}}
P_x_and_y = P_x_given_y * P_y                                   # P(x,y) = P(x|y) * P(x)

def log2(p):
    """ Use clipping to avoid -infinity in log2 """
    th = 0.000000001
    p[p < th] = th
    return np.log2(p)

As binary search splits the search space into two equiprobable parts¹ and essentially asks which of the two parts $x$ is in, the entropy of the answer is $H_\text{b}(0.5) = 1 \,\text{bit}$. In our case subsequent questions are independent, hence, we gain $1\,\text{bit}$ of information with each question. Therefore, we should expect the number of questions to equal the ammount of information we want to gain in $\text{bit}$.

For now we want to find the value $X$, i.e., the number of required questions should equal $H(X)$ in $\text{bit}$.

<small>¹ This is only exactly true if the size of the search space is a power of two. If it is not, a perfect split is not possible in every step and we gain less information in the steps where the two splits are unequal.</small>

In [ ]:
src.resample()

# Compute H(X):

H_X = - np.sum(P_x * log2(P_x))         # H(X) = - sum_{x = 0..M-1} P(x) * log2(P(x))

print(f'We need H(X) = {H_X:.2f} bit of information to know X = x!')
print()

# Now compare to the number of questions we need with binary search:

possible_x = np.arange(src.M) # np.arange(M) produces an array of numbers from 0 to M-1

num_questions = binary_search(src, possible_x)
print(f'Finding x took {num_questions} (binary) questions.')

Knowing $Y$ reduces the uncertainty about $X$. We expect to only need $H(X|Y)\,\text{bit}$ of information (i.e., $H(X|Y)$ questions) to find $X=x$ if we have $Y$.

In [ ]:
src.resample()

# Compute H(X|Y)

H_XgY = - np.sum(P_x_and_y * log2(P_x_given_y))   # H(X|y) = - sum_{y = 0..M-1} sum_{x in X(y)} P(x,y) * log2(P(x|y))


print(f'We need H(X|Y) = {H_XgY:.2f} bit of information to know X = x if we know Y!')
print()

# Now compare to the number of questions we need with binary search:

y = src.observe_y()
delta = np.arange(src.m)

possible_x = (y - delta) % src.M
print(f'We know y = {y}, hence, x is in {possible_x}.')

num_questions = binary_search(src, possible_x)
print(f'Finding x took {num_questions} (binary) questions.')

The reduction in uncertainty about $X$ gained by observing $Y$ is the mutual information $I(X;Y) = H(X) - H(X|Y)$.

In [ ]:
src.resample()

# Compute I(X;Y)

I_XY = np.sum(P_x_and_y * log2(P_x_and_y / (P_x * P_y)))   # H(X|y) = sum_{y = 0..M-1} sum_{x in X(y)} P(x,y) * log2(P(x,y) / (P(x) P(y)))


print(f'The mutual information between X and Y is I(X;Y) = {I_XY:.2f} bit!')
print()

# Now compare to the number of questions we need with binary search:

print('Without knowing Y:\n', end='  ')
possible_x = np.arange(src.M)
num_questions_wo_knowing_y = binary_search(src, possible_x)
print(f'  Finding x took {num_questions_wo_knowing_y} (binary) questions.')
print()

y = src.observe_y()
delta = np.arange(src.m)

print(f'With knowing y = {y}:\n', end='  ')
possible_x = (y - delta) % src.M

num_questions_knowing_y = binary_search(src, possible_x)
print(f'  Finding x took {num_questions_knowing_y} (binary) questions.')
print()

print(f'Knowing Y reduces the number of necessary questions by {num_questions_wo_knowing_y - num_questions_knowing_y}!')